# Denoising mel-spectrograms with a residual dense network

## Setup

### Imports

In [2]:
from audio_denoising.data.loader import load

### Variables

In [3]:
TRAIN_DATASET = "dataset/train"
VAL_DATASET = "dataset/val"

In [5]:
d = load(TRAIN_DATASET)

In [8]:
for i in d:
    print(i)

RuntimeError: "add_cpu/sub_cpu" not implemented for 'Half'